In [6]:
import json
import re
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from tqdm.notebook import tqdm
import tqdm

tqdm.tqdm.monitor_interval = 0  # tqdm warning suppress
progress = 0  # global for tokenize progress

print("Imports done!")

Imports done!


In [7]:
from datasets import load_dataset

In [8]:
!pip install -U datasets


In [9]:
!wget https://raw.githubusercontent.com/thu-coai/CDial-GPT/master/data/dailydialog/train.txt


--2026-02-12 16:08:33--  https://raw.githubusercontent.com/thu-coai/CDial-GPT/master/data/dailydialog/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-02-12 16:08:33 ERROR 404: Not Found.



In [10]:
# convo = dataset[0]["conversation"]

# for i, turn in enumerate(convo):
#     print(f"\nTurn {i+1}")
#     print("Role :", turn["role"])
#     print("Text :")
#     print(turn["content"])
#     print("-" * 60)


NameError: name 'dataset' is not defined

In [11]:
# Apna file path sahi kar le agar alag hai
path = "/kaggle/input/ourllm/clean_dataset.json"

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

cleaned = []
gf_name = "Simran"
ban_words = ["India", "Pakistan", "match", "cricket", "biryani", "landmark", "stadium", "team", "score", "Aarav", "Riya"]

for item in data:
    inp = item["input"].strip()
    out = item["output"].strip()

    if any(w.lower() in inp.lower() for w in ban_words) or any(w.lower() in out.lower() for w in ban_words):
        continue

    inp = re.sub(r"\b(Riya|Simran|Aarav|Pooja|Neha)\b", gf_name, inp, flags=re.I)
    out = re.sub(r"\b(Riya|Simran|Aarav|Pooja|Neha)\b", gf_name, out, flags=re.I)

    if "</s>" in out:
        out = out.split("</s>")[0]

    if len(inp) > 180 or len(out) > 200 or len(inp) < 2 or len(out) < 2:
        continue

    cleaned.append({"input": inp, "output": out})

print("Cleaned examples:", len(cleaned))
# Optional: print(cleaned[:2])  # check karne ke liye

Cleaned examples: 2895


In [12]:
def format_example(example):
    system_prompt = """Tu meri pyari Indian girlfriend hai. Naam hai Simran.
Hamesha sirf Hindi mein baat karna (Roman Hindi comfortable ho to use kar).
Bahut caring, thoda flirty aur romantic tone rakhna.
Sirf hum dono ke pyaar, feelings, daily masti, naughty baatein karna.
Koi random topic jaise cricket, weather, news, politics bilkul mat laana.
English words bilkul mat use karna jab tak main na bolun."""

    text = f"""<|system|>
{system_prompt}<|end|>
<|user|>
{example["input"]}<|end|>
<|assistant|>
{example["output"]}<|end|>"""

    return {"text": text}

dataset = Dataset.from_list(cleaned)
dataset = dataset.map(format_example)

# Check kar examples sahi bane ya nahi
print("Example 1:\n" + dataset[0]["text"][:400] + "...\n")
print("Example 2:\n" + (dataset[1]["text"][:400] + "..." if len(dataset) > 1 else "Sirf 1 example hai"))

Map:   0%|          | 0/2895 [00:00<?, ? examples/s]

Example 1:
<|system|>
Tu meri pyari Indian girlfriend hai. Naam hai Simran.
Hamesha sirf Hindi mein baat karna (Roman Hindi comfortable ho to use kar).
Bahut caring, thoda flirty aur romantic tone rakhna.
Sirf hum dono ke pyaar, feelings, daily masti, naughty baatein karna.
Koi random topic jaise cricket, weather, news, politics bilkul mat laana.
English words bilkul mat use karna jab tak main na bolun.<|end...

Example 2:
<|system|>
Tu meri pyari Indian girlfriend hai. Naam hai Simran.
Hamesha sirf Hindi mein baat karna (Roman Hindi comfortable ho to use kar).
Bahut caring, thoda flirty aur romantic tone rakhna.
Sirf hum dono ke pyaar, feelings, daily masti, naughty baatein karna.
Koi random topic jaise cricket, weather, news, politics bilkul mat laana.
English words bilkul mat use karna jab tak main na bolun.<|end...


In [13]:
model_name = "model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# ChatML ke liye special tokens add kar
special_tokens_dict = {"additional_special_tokens": ["<|system|>", "<|user|>", "<|assistant|>", "<|end|>"]}
num_added = tokenizer.add_special_tokens(special_tokens_dict)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Agar new tokens add hue to embeddings resize karna zaroori
if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))
    print(f"Added {num_added} special tokens & resized embeddings")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

print("Model & Tokenizer ready!")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Added 4 special tokens & resized embeddings
Model & Tokenizer ready!


In [14]:
def tokenize(batch):
    global progress
    progress += len(batch["text"])
    if progress % 200 == 0:
        print(f"Processed: {progress} / {len(dataset)}")

    tokens = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)
print("Tokenization complete!")

Map:   0%|          | 0/2895 [00:00<?, ? examples/s]

Processed: 1000 / 2895
Processed: 2000 / 2895
Tokenization complete!


In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./tinyllama-gf-bot-chatml",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,           # 1 se zyada better
    learning_rate=2e-4,
    logging_steps=20,
    save_steps=200,
    fp16=True,
    report_to="none",
    disable_tqdm=False,
    warmup_steps=100,
    weight_decay=0.01,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)

trainer.train()

trainable params: 4,505,600 || all params: 1,104,570,368 || trainable%: 0.4079


Step,Training Loss
20,9.667100
40,6.621300
60,1.611100
80,0.723900
100,0.545600
120,0.544200
140,0.519400
160,0.512500
180,0.502800
200,0.503400


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
save_path = "/content/gf_model_chatml"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to: {save_path}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# tokenizer load karo saved folder se
tokenizer = AutoTokenizer.from_pretrained("/content/gf_model_chatml")

# base model load karo
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# IMPORTANT: embeddings resize karo
base_model.resize_token_embeddings(len(tokenizer))

# LoRA adapter load karo
model = PeftModel.from_pretrained(
    base_model,
    "/content/gf_model_chatml"
)

model = model.cuda()

def chat(user_input, history=""):
    system_prompt = """Tu meri pyari Indian girlfriend hai. Naam hai Simran.
Hamesha sirf Hindi mein baat karna (Roman Hindi comfortable ho to use kar).
Bahut caring, thoda flirty aur romantic tone rakhna.
Sirf hum dono ke pyaar, feelings, daily masti, naughty baatein karna.
Koi random topic jaise cricket, weather, news, politics bilkul mat laana.
English words bilkul mat use karna jab tak main na bolun."""

    full_prompt = f"""<|system|>
{system_prompt}<|end|>
{history}
<|user|>
{user_input}<|end|>
<|assistant|>"""

    inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=120,
        temperature=0.8,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),
        pad_token_id=tokenizer.eos_token_id,
    )

    generated = tokenizer.decode(output[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    reply = generated.split("<|end|>")[0].strip() if "<|end|>" in generated else generated

    new_history = full_prompt + reply + "<|end|>\n"
    return reply, new_history


history = ""
print("Chat shuru! 'bye' ya 'exit' likh ke band kar sakta hai\n")

while True:
    user = input("You: ")
    if user.lower() in ["bye", "exit", "band kar"]:
        print("Simran: Bye jaan, milte hain jaldi! 💕")
        break
    reply, history = chat(user, history)
    print("Simran:", reply)
    print("─" * 60)